This is the torch to tensorrt converter notebook. I will add more stuff once I know what to add. I am following this link: https://learnopencv.com/how-to-convert-a-model-from-pytorch-to-tensorrt-and-speed-up-inference/

In [13]:
import torch
import os
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn

In [14]:
trained_model_path = "../runs/traversky/2023_03_05_21_30_33/model.pth"
ONNX_FILE_PATH = 'bin_classifier.onnx'

In [17]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(2048, 2)

In [18]:
# model loaded
checkpoint = torch.load(trained_model_path)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

Now the tensorrt conversion. 

First step is to make .onnx file.  

In [22]:
dummy_input = torch.empty((1, 3, 256, 256))
torch.onnx.export(model, dummy_input, ONNX_FILE_PATH, input_names=['input'],
                  output_names=['output'], export_params=True)

Now lets see the tensor rt

In [23]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt as trt

ModuleNotFoundError: No module named 'pycuda'